#Итоговый проект

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from os import listdir
import codecs
import pickle                                                                   
from bs4 import BeautifulSoup as BS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from boilerpipe.extract import Extractor
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse import hstack
from scipy.sparse import vstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
%matplotlib inline

In [ ]:
submit = False

In [ ]:
train_data = pd.read_csv('train_groups.csv',dtype=np.int16)

In [ ]:
#это парсинг теста
if submit:
    test_data= pd.read_csv('test_groups.csv',dtype=np.int16)

    parser_type = 'NumWordsRulesExtractor'
    def save_obj(obj, name ):                                                       
        with open(name + '.pkl', 'wb') as f:                                        
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            

    def load_obj(name ):                                                            
        with open(name + '.pkl', 'rb') as f:                                        
            return pickle.load(f)                                                   

    path = 'content/'                                                               
    texts = {}                                                                      
    print('Start: ' + parser_type)                                                  
    test_data = pd.read_csv('test_groups.csv',dtype=np.int16)                      
    #for filename in ['6770.dat']:                                                  
    for filename in tqdm(listdir(path)):                                            
        doc_id = int(filename.strip('.dat'))                                        
        if doc_id not in test_data.doc_id.values:                                  
            continue                                                                
        try:                                                                        
            with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
                url = f.readline().strip()                                          
                html = f.read()                                                     
                extractor = Extractor(extractor=parser_type, html=html)             
                s = extractor.getText()                                             
                s=s.replace('\n'," ")                                               
                s=s.replace('\t'," ")                                               
                s=s.replace('\r'," ")                                               
                texts[doc_id] = s                                                   
        except UnicodeDecodeError: 
            print('error {}'.format(filename))                                      
            with codecs.open(path + filename, 'r', 'utf-8') as f:                   
                url = f.readline().strip()                                          
                html = f.read()                                                     
            bs = BS(html, 'html.parser')#ищем след страницу                         
            for s in bs.select('script'):                                           
                s.extract()                                                         
            s = bs.get_text()                                                       
            s=s.replace('\n',"")                                                    
            s=s.replace('\t',"")                                                    
            s=s.replace('\r',"")                                                    
            texts[doc_id] = s                                                       
    test_data['text'] = test_data.apply(lambda row: texts[row.doc_id], axis = 1)  


In [ ]:
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)       

In [6]:
name = 'NumWordsRulesExtractor'
name = 'train_data'+name
train_data = load_obj(name)#Загружаем готовый распарсенный текст уже

In [7]:
doc_to_title = {}
with open('docs_titles.tsv', encoding = 'UTF-8') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
doc_titles = []
for id in train_data.doc_id.values:
    doc_titles.append(doc_to_title[id])
train_data['title'] = doc_titles
if submit:
    test_doc_titles = []
    for id in test_data.doc_id.values:
        test_doc_titles.append(doc_to_title[id])
    test_data['title'] = test_doc_titles
print(len(doc_titles))


11690


In [22]:
def get_splits(data, n=3, random_state=None):
    groups = train_data.group_id.unique()
    kf = KFold(n_splits=n, shuffle=True, random_state=random_state)
    for i,j in kf.split(groups):
        train, test = data.group_id.isin(groups[i].reshape(-1)), data.group_id.isin(groups[j].reshape(-1))
        yield (train[train].index.values, test[test].index.values)

In [10]:
#vectorizer = CountVectorizer(ngram_range = (1,3))#max_features = 70)
vectorizer = CountVectorizer()#max_features = 70)

In [13]:
all_groups = np.array(list(range(1,310))).reshape(-1,1)
i = OneHotEncoder()
i.fit(all_groups)
groups_one_hot_train = i.transform(train_data.group_id.values.reshape(-1,1))
if submit:
    groups_one_hot_test   = i.transform(test_data.group_id.values.reshape(-1,1))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [19]:
def get_features(data):
    scaler = StandardScaler(with_mean=False)
    number = 1
    for group in tqdm(data.group_id.unique()):
        number +=1
        elements_in_same_group = data.group_id == group

        vectorizer.fit(data.title[elements_in_same_group] + data.text[elements_in_same_group])
        texts_vectorized_titles = vectorizer.transform(data.title[elements_in_same_group])
        texts_vectorized_titles = scaler.fit_transform(texts_vectorized_titles)

        texts_vectorized_texts = vectorizer.transform(data.text[elements_in_same_group])
        texts_vectorized_texts = scaler.fit_transform(texts_vectorized_texts)

        dist_matr = pairwise_distances(texts_vectorized_titles, texts_vectorized_titles, metric = 'cosine')
        dist_matr2 = pairwise_distances(texts_vectorized_texts, texts_vectorized_texts, metric = 'cosine')
        dist_matr3 = pairwise_distances(texts_vectorized_texts, texts_vectorized_titles, metric = 'cosine')

        #clustering = DBSCAN(eps = 1.3, min_samples=5).fit(dist_matr)
        #clf = IsolationForest(random_state=0).fit_predict(dist_matr)

        dist_matr_sorted = np.sort(dist_matr,axis=1)[:,0:21]
        dist_matr_sorted2 = np.sort(dist_matr2,axis=1)[:,0:21]
        dist_matr_sorted3 = np.sort(dist_matr3,axis=1)[:,0:21]
    #     dist_matr_sorted = np.sort(dist_matr,axis=1)[:,0:min(50,dist_matr.shape[1])]
    #     dist_matr_sorted2 = np.sort(dist_matr2,axis=1)[:,0:min(50,dist_matr2.shape[1])]
    #     dist_matr_sorted3 = np.sort(dist_matr3,axis=1)[:,0:min(50,dist_matr3.shape[1])]

    #     if dist_matr_sorted.shape[1] < 50:
    #         dist_matr_sorted = np.hstack((dist_matr_sorted, np.tile(dist_matr_sorted[:,-1:], (1, 50-dist_matr_sorted.shape[1]))))
    #         dist_matr_sorted2 = np.hstack((dist_matr_sorted2, np.tile(dist_matr_sorted2[:,-1:], (1, 50-dist_matr_sorted2.shape[1]))))
    #         dist_matr_sorted3 = np.hstack((dist_matr_sorted3, np.tile(dist_matr_sorted3[:,-1:], (1, 50-dist_matr_sorted3.shape[1]))))


        mean_dist = dist_matr.mean(axis = 1)
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted2))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted3))
       # dist_matr_sorted = np.hstack((dist_matr_sorted, clustering.labels_.reshape(-1,1)))
    #   dist_matr_sorted = hstack((dist_matr_sorted, texts_vectorized))

        if number == 2 :
            features_by_dist =  dist_matr_sorted
        else:
            features_by_dist = vstack((features_by_dist, dist_matr_sorted))
    return features_by_dist.tocsr()

In [20]:
if submit:
    features_by_dist = get_features(test_data)
    features_by_dist = hstack([features_by_dist, groups_one_hot_test])
    features_by_dist = features_by_dist.tocsr()
else:
    target = train_data.target.values
    features_by_dist = get_features(train_data)
    features_by_dist = hstack([features_by_dist, groups_one_hot_train])
    features_by_dist = features_by_dist.tocsr()

In [27]:
#VALIDATION
max_score = 0
scores = []
for i in tqdm(range(5)):
    for train, test in get_splits(train_data,n=3,random_state=i):
        model = RandomForestClassifier(n_jobs=6,n_estimators=200)
        model.fit(features_by_dist[train], target[train])
        pred = (model.predict(features_by_dist[test]))               
        score = f1_score(pred,target[test])
        scores.append(score)
        print(score, end=' ')
    print()

print('\nAvg score:' ,sum(scores)/len(scores))

0.723110151187905 0.7288765088207985 0.6989583333333333 
0.7031476997578692 0.7317073170731707 0.6998011928429423 
0.7544075440754408 0.7258150721539285 0.6407480314960631 
0.7545893719806763 0.6940718303764605 0.6893056259503295 
0.6771724448446645 0.7218119153126539 0.7543447627994364 


Avg score: 0.713191186800378


In [ ]:
#SAVE model
from sklearn.ensemble import RandomForestClassifier 
model = RandomForestClassifier(n_jobs=6,n_estimators=200)
model.fit(features_by_dist, target)
save_obj(model, 'model')

In [ ]:
#SUBMIT
from sklearn.ensemble import RandomForestClassifier 
print(type(features_by_dist))
model = load_obj('model') 
pred = (model.predict(features_by_dist)               
pred.shape

In [ ]:
test_data['target'] = pred
test_data.to_csv('sub',index=False,columns=['pair_id','target'])